In [ ]:
1+1

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')

In [ ]:
#pip show langchain

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

In [ ]:
'''text='what is 1+1?'

output = chat.invoke(text)
print(output)'''

In [ ]:
prompt = """根據以下上下文回答問題。
如果無法根據提供的訊息回答，請回答"我不知道"。

上下文: 大語言模型(LLM)是自然語言處理中最新使用的模型。
與較小的模型相比，他們出色的性能使他們對於建構支持自然語言處理的應用程式的開發人員非常有用。
這些模型可以通過Hugging Face的transformers庫、OpenAI的openai庫以及Cohere的cohere庫來訪問。

問題: 什麼庫提供LLM?

回答:
"""

In [ ]:
result = chat.invoke(prompt)
print(result.content)

In [ ]:
from langchain import PromptTemplate

In [ ]:
template = """根據以下上下文回答問題。
如果無法根據提供的訊息回答，請回答"我不知道"。

上下文: 大語言模型(LLM)是自然語言處理中最新使用的模型。
與較小的模型相比，他們出色的性能使他們對於建構支持自然語言處理的應用程式的開發人員非常有用。
這些模型可以通過Hugging Face的transformers庫、OpenAI的openai庫以及Cohere的cohere庫來訪問。

問題: {query}

回答:
"""

In [ ]:
prompt_template = PromptTemplate(
    input_variables = ["query"], 
    template = template
)

In [ ]:
question = "LLM是什麼"
myprompt = prompt_template.format(query = question)
print(chat.invoke(myprompt).content)

In [ ]:
prompt = """如下是AI駐守的對話。
這位助手比較幽默，對用戶的提問會給出非常有創意和有趣的回應。
以下是一些例子:

用戶: 你好嗎?
AI: 我的感覺很不錯，你呢?

用戶: 現在幾點了?
AI: 你等等，我要先解鎖我的iphone

用戶: 什麼味道的冰淇淋好吃?
AI:
"""

In [ ]:
print(chat.invoke(prompt))

In [ ]:
from langchain import FewShotPromptTemplate

In [ ]:
examples = [
    {
        "query":"你好嗎?",
        "answer":"我的感覺不錯，你呢?"
    },
    {
        "query":"現在幾點?",
        "answer":"你等等,我要解鎖一下iphone"
    }
]

In [ ]:
example_template = """
    User: {query}
    AI: {answer}
"""

In [ ]:
example_prompt = PromptTemplate(
    input_variables = ["query", "answer"],
    template = example_template
)

In [ ]:
prefix = """"
    如下是一位AI助手的對話。
    他總會用幽默的回覆方式回應用戶。
    以下是助手對話的一些例子
"""

In [ ]:
suffix = """
    用戶: {userQuery}
    AI:
"""

In [ ]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["userQuery"],
    example_separator = "\n"
)

In [ ]:
answer = "如何讓生活更加幸福?"
formatted_prompt = few_shot_prompt_template.format(userQuery=answer)
print(formatted_prompt)

In [ ]:
print(chat.invoke(formatted_prompt).content)

In [ ]:
#課程2-9 chain
template = """
    回答如下問題，並說出準確的計算答案

    問題: {query}
    AI:
"""

In [ ]:
from langchain import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=["query"]
)

In [ ]:
from langchain.chains.llm import LLMChain

chain = LLMChain(
    llm=chat,
    prompt=prompt_template,
    verbose=True
)

In [ ]:
oputput = chain.invoke({"query":"18的0.6352次方式多少?"})
print(oputput["text"])

In [ ]:
from langchain.chains import LLMMathChain

In [ ]:
math_chain = LLMMathChain.from_llm(
    llm=chat,
    verbose=True
)

In [ ]:
result = math_chain.invoke({"question":"18的0.6352次方式多少?"})

In [ ]:
print(math_chain.prompt.template)

In [ ]:
import re
def transform_func(inputs: dict):
    text = inputs["tr_text"]

    text = re.sub(r'[ ]+', ' ', text)

    return {"tr_output": text}

In [ ]:
from langchain.chains.transform import TransformChain

In [ ]:
clean_spaces_chain = TransformChain(
    input_variables=["tr_text"],
    output_variables=["tr_output"],
    transform=transform_func
)

In [ ]:
output = clean_spaces_chain.invoke({"tr_text": "   Hello                                              World!~                   "})    

print(output["tr_output"])

In [ ]:
template = """
    轉換如下文字:
    
    {tr_output}
    
    新形式{cvr_style}
    
    轉換:
"""

In [ ]:
prompt = PromptTemplate(
    #input_variables=["cvr_output", "cvr_style"],
    input_variables=["cvr_style"],
    template=template
)

In [ ]:
convert_chain = LLMChain(
    llm=chat,
    prompt=prompt,
    output_key="new_output",
    verbose=True
)

In [ ]:
# new_output = convert_chain.invoke({"cvr_output": "Gong Hey Fat Choy", "cvr_style": "變中文"})
# print(new_output["new_output"])

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
sequential_chain = SequentialChain(
    chains = [clean_spaces_chain, convert_chain],
    input_variables=["tr_text", "cvr_style"],
    output_variables=["new_output"]
)

In [ ]:
input_text = """
Gong Hey Fat Choy               is a             traditional                        Cantonese phrase                used to convey          well-wishes during                        the Chinese             New Year celebration."""

In [ ]:
output = sequential_chain.invoke({"tr_text": input_text, "cvr_style": "轉換成中文"})
print(output["new_output"])

In [ ]:
#2-10 記憶(把過去資料存下來，問問題會再丟給LLM)
from langchain.chains import ConversationChain

In [ ]:
conversation = ConversationChain(
    llm=chat
)

In [ ]:
print(conversation.prompt.template)

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
#所有記憶都存下來
conversation_buf = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

In [ ]:
response = conversation_buf.invoke({"input": "你知道什麼是TikTok嗎?"})
print(response)

In [ ]:
response = conversation_buf.invoke({"input": "你知道如何在Google Chrome上面添加Tab Group嗎?"})
print(response)

In [ ]:
print(conversation_buf.memory.buffer)

In [ ]:
from langchain.memory import ConversationSummaryMemory  

In [ ]:
#每次都把過去記憶做總結，再存下來
conversation_sum = ConversationChain(
    llm=chat,
    memory=ConversationSummaryMemory(llm=chat)
)

In [ ]:
print(conversation_sum.memory.prompt.template)

In [ ]:
conversation_sum.invoke({"input": "你知道什麼是TikTok嗎?"})

In [ ]:
conversation_sum.invoke({"input": "你知道如何在Google Chrome上面添加Tab Group嗎?"})

In [ ]:
print(conversation_sum.memory.buffer)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
#可以設定要記錄前幾個window的紀錄
conversation_buff_win = ConversationChain(
    llm=chat,
    memory=ConversationBufferWindowMemory(k=1)
)

In [ ]:
conversation_buff_win.invoke({"input": "你知道如何在Google Chrome上面添加Tab Group嗎?"})

In [ ]:
conversation_buff_win.invoke({"input": "你知道如何在Youtube實現畫中畫功能嗎?"})

In [ ]:
conversation_buff_win.invoke({"input": "你知道如何做pizza嗎?"})